In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb

In [ ]:
%cd drive
%cd 'My Drive'
%cd 'RL'
%cd 'Homework'
%cd 'Homework 4'

In [ ]:
'''Needed Libraries and Environment'''
import Environment
import Modified_Environment
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from collections import defaultdict
import sys
import itertools
import matplotlib
import pandas as pd
import pickle

'''Constants Defined'''
PROBABILITY = 0
STATE = 1
ACTION = 2
REWARD = 3
MAX_T = 10000
TOTAL_EPISODES = 2000
gamma = 0.9
ALPHA = 0.2
MIN = 0.1
DECAY_FACTOR = 0.99

In [ ]:
class SarsaAgent():
  
    def __init__(self, env, gamma):
      self.env = env
      self.Q = defaultdict(lambda: np.zeros(env.action_space.n))
      self.actions = env.action_space.n
      self.gamma = gamma
      self.sarsa_episode_scores = []

    def epsilon_greedy_policy(self, state, epsilon):
        epsilon_policy = np.ones(self.actions, dtype=float) * epsilon / self.actions
        best_action = np.argmax(self.Q[state])
        epsilon_policy[best_action] += (1.0 - epsilon)
        return epsilon_policy

    def reach_island(self):
        #epsilon = 1 #<-- uncomment if you want to use decaying epsilon 
        #epsilon = 0.3 #<-- uncomment if you want to use constant epsilon
        for episode in range(1, TOTAL_EPISODES + 1):
            episode_score = 0
            if episode % 2 == 0:
              #epsilon = max(epsilon * DECAY_FACTOR, MIN) #<-- uncomment if you want to use decaying epsilon
              print("\rEpisode {}/{}.".format(episode, TOTAL_EPISODES), end="")
              sys.stdout.flush()
            state = self.env.reset()
            done = False
            probs = self.epsilon_greedy_policy(state, epsilon)
            action = np.random.choice(np.arange(len(probs)), p = probs)
            for t in itertools.count():
                next_state, reward, done, _ = self.env.step(action)
                next_probs = self.epsilon_greedy_policy(next_state, epsilon)
                next_action = np.random.choice(np.arange(len(next_probs)), p = next_probs)
                episode_score += reward
                information, next_information = [probs, state, action, reward], [next_probs, next_state, next_action]
                self.sarsa_returns(information, next_information)
                if done or t < MAX_T : break
                state, action = next_state, next_action
            self.sarsa_episode_scores.append(episode_score)

    def sarsa_returns(self, info, next_info):
        target = info[REWARD] + self.gamma * self.Q[next_info[STATE]][next_info[ACTION]]
        self.Q[info[STATE]][info[ACTION]] += ALPHA * (target - self.Q[info[STATE]][info[ACTION]])

In [ ]:
# env =  Environment.GridworldEnv() #<--uncomment for normal environment
# env =  Modified_Environment.GridworldEnv() # <--uncomment for modified environment (bonus part)
agent = SarsaAgent(env, gamma)
agent.reach_island()

Episode 2000/2000.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(TOTAL_EPISODES), agent.sarsa_episode_scores, color = '#633974', label = '$\epsilon$ = from $1$ to $0.1$')
plt.xlabel('Episodes ->')
plt.ylabel('Score ->')
plt.title('SARSA')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12 ,6))
window = 50
smoothed_score = pd.Series(agent.sarsa_episode_scores).rolling(window , min_periods = window).mean()
plt.plot(smoothed_score, color = '#633974', label = '$\epsilon$ = from $1$ to $0.1$')
plt.xlabel("epsiode ->")
plt.ylabel("epsiode score (smoothed) -> ")
#plt.annotate(smoothed_score.iloc[-1], xy=(2000, -14), xytext=(2000, -14), color = '#633974',) # <--uncomment for modified environment (bonus part)
plt.title('SARSA')
plt.legend()
plt.show()

In [ ]:
#i wrote my reward data into a file to use it later

# with open('sarsa_episode_scores', 'wb') as fp:
#     pickle.dump(agent.sarsa_episode_scores, fp)